# Save courses to RDF

## Read data from Firestore

In [1]:
# init
from os import environ
environ['FIRESTORE_EMULATOR_HOST'] = 'localhost:8080'
environ['FIREBASE_AUTH_EMULATOR_HOST'] = 'localhost:8083'
environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../serviceAccountKey.json'

from firebase_admin import firestore, initialize_app, credentials
cred = credentials.ApplicationDefault()
initialize_app(cred)
db = firestore.client()

In [9]:
# get a program
siit_ref = db.collection(u'programs').document(u'siit')
program = siit_ref.get().to_dict()
program['naziv']

'softversko inzenjerstvo i informacione tehnologije'

In [17]:
# get courses
courses_snapshot = siit_ref.collection(u'courses').stream()
courses = [course.to_dict() for course in courses_snapshot]
courses[0]['naziv']

'Završni rad - izrada i odbrana'

## Write courses to rdf

Simple rdflib example

In [33]:
from rdflib import *

# create Bob & Linda
bob = URIRef('http://example.org/people/Bob')
linda = BNode()

# create Literals
name = Literal('Bob')
age = Literal(24)
height = Literal(180)

# create graph & add namespace
g = Graph()
g.bind('foaf', FOAF)

# add Bob & Linda to the graph
g.add((bob, RDF.type, FOAF.Person))
g.set((bob, FOAF.name, name))
g.set((bob, FOAF.age, age))
g.add((bob, FOAF.knows, linda))
g.add((linda, RDF.type, FOAF.Person))
g.set((linda, FOAF.name, Literal('LINDA')))

# write graph to file
g.serialize(destination='program.ttl')

# print graph, print values in graph
print(g.serialize(), end='\n-----------\n')
print(g.value(bob, FOAF.age))

# remove all triples about bob
g.remove((bob, None, None)) 
print(g.serialize(), end='\n-----------\n')

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/Bob> a foaf:Person ;
    foaf:age 24 ;
    foaf:knows [ a foaf:Person ;
            foaf:name "LINDA" ] ;
    foaf:name "Bob" .


-----------
24
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

[] a foaf:Person ;
    foaf:name "LINDA" .


-----------


In [94]:
from re import compile
def date_to_xsd_date(string_with_date):
    p = compile('(\d{2}).(\d{2}).(\d{4}).')
    day, month, year = p.search(string_with_date).groups()
    return f'{year}-{month}-{day}'

In [107]:
g = Graph()
g.bind('dc', DC)
g.bind('foaf', FOAF)
g.bind('aiiso', 'http://purl.org/vocab/aiiso/schema#')

In [108]:


for course in courses[30:32]:
    course_node = URIRef(course['url'])

    aiiso_course = URIRef('http://purl.org/vocab/aiiso/schema#Course')
    g.add((course_node, RDF.type, aiiso_course))
    g.add((course_node, DC.title, Literal(course['naziv'])))
    g.add((course_node, DC.description, Literal(course['sadrzaj'])))
    g.add((course_node,
           DC.date,
           Literal(date_to_xsd_date(course['osnovneInformacije']), datatype=XSD.date)))



print(g.serialize(format='turtle'))
# print(g.serialize(format='turtle', destination='courses.ttl'))


@prefix aiiso: <http://purl.org/vocab/aiiso/schema#> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.ftn.uns.ac.rs/1385388292/masinsko-ucenje> a aiiso:Course ;
    dc:date "2012-11-16"^^xsd:date ;
    dc:description "(1) Nadgledano obučavanje:  Linearna regresija (jednostruka i višestruka regresija, metod gradijentnog spusta i njegove varijante, analitičko rešenje u zatvorenoj formi, uticaj outlier-a); Neparametarski pristup (metod k najbližih suseda i krernel regresija);  Metod maksimalne verodostojnosti; Klasifikacija (logistička regresija, perceptron, naivni Bajes, metod potpornih vektora, ansambli klasifikatora).  (2) Pravilan dizajn eksperimenata i selekcija optimalnog modela: adekvatne mere performanse za različite probleme; eksperimentalni postupci (unakrsna validacija, podela na trening/validacioni/test skup, adekvatna selekcija i optimizacija modela); preprilagođavanje i regularizacija (grebena regresija, laso 

In [103]:
professors = courses[30]['izvodjaciNastave']
professor_node = BNode()
g.add((professor_node, RDF.type, FOAF.Person))
g.add((professor_node, FOAF.name, Literal(professors[0]['punoIme'])))

<Graph identifier=N7cc4d7ef53804551b98ea3c313db17c4 (<class 'rdflib.graph.Graph'>)>

In [106]:
aiiso_teaches = URIRef('http://purl.org/vocab/aiiso/schema#teaches')
g.add((professor_node, aiiso_teaches, course_node))

print(g.serialize())

@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix ns1: <http://purl.org/vocab/aiiso/schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.ftn.uns.ac.rs/1385388292/masinsko-ucenje> a ns1:Course ;
    dc:date "2012-11-16"^^xsd:date ;
    dc:title "Mašinsko učenje" .

<http://www.ftn.uns.ac.rs/n1108388269/softverski-obrasci-i-komponente> a ns1:Course ;
    dc:date "2007-10-01"^^xsd:date ;
    dc:title "Softverski obrasci i komponente" .

[] a foaf:Person ;
    ns1:teaches <http://www.ftn.uns.ac.rs/n1108388269/softverski-obrasci-i-komponente> ;
    foaf:name "Kovačević dr Aleksandar" .


